<a href="https://colab.research.google.com/github/Luana-lrb/LIPAI_Python/blob/main/onboarding/src/semana_8/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Redes Neurais Convolucinais com PyTorch**

### Como base foi utilizado o trabalho dos Profs. Storopoli & Souza disponível no Google Colab:
https://colab.research.google.com/github/storopoli/ciencia-de-dados/blob/main/notebooks/Aula_19_Redes_Neurais_Convolucionais_com_PyTorch.ipynb#scrollTo=oR24U2WYKVrw


Importando bibliotecas necessárias:

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

1) Device: usa GPU se disponível

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Treinando em:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

Treinando em: cuda
GPU: Tesla T4


Transform e datasets:

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

root_path = '/home/storopoli/Downloads'  # Está funcionando assim
train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=trans, download=True)
test_dataset  = torchvision.datasets.MNIST(root=root_path, train=False, transform=trans)


100%|██████████| 9.91M/9.91M [00:00<00:00, 20.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 490kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.73MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.8MB/s]


DataLoaders: use pin_memory quando usar GPU para acelerar cópias

In [5]:
batch_size = 32
num_workers = 0  # ajuste conforme sua máquina; no Windows usar 0 às vezes
pin_memory = True if device.type == "cuda" else False

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=pin_memory)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, pin_memory=pin_memory)


Modelo

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(7 * 7 * 64, 1000),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

Instancia e move o modelo para device ANTES de criar o otimizador

In [7]:
model = ConvNet().to(device)
print(model)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Parâmetros treináveis:", count_parameters(model))

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=3136, out_features=1000, bias=True)
    (1): ReLU()
  )
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)
Parâmetros treináveis: 3199106


 Loss, hyperparams e otimizador (otimizador depois do model.to)

In [8]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
epochs = 6
optimizer = Adam(model.parameters(), lr=learning_rate)
torch.backends.cudnn.benchmark = True if device.type == "cuda" else False

Loop de treino com envio explícito de tensores para device

In [9]:
total_step = len(train_loader)
loss_list = []
acc_list = []

for epoch in range(epochs):
    model.train()  # importante: habilita treinamento (dropout, batchnorm, etc.)
    for i, (images, labels) in enumerate(train_loader):
        # envia dados para a GPU/CPU
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # forward
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss_list.append(loss.item())

        # backward + step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # acurácia no batch
        total = labels.size(0)
        predicted = outputs.argmax(dim=1)  # forma recomendada
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print(f"Época [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], "
                  f"Custo: {loss.item():.4f}, Acurácia(batch): {100 * (correct / total):.3f}%")

Época [1/6], Step [100/1875], Custo: 0.2033, Acurácia(batch): 96.875%
Época [1/6], Step [200/1875], Custo: 0.0220, Acurácia(batch): 100.000%
Época [1/6], Step [300/1875], Custo: 0.2542, Acurácia(batch): 87.500%
Época [1/6], Step [400/1875], Custo: 0.2974, Acurácia(batch): 93.750%
Época [1/6], Step [500/1875], Custo: 0.2490, Acurácia(batch): 93.750%
Época [1/6], Step [600/1875], Custo: 0.0550, Acurácia(batch): 96.875%
Época [1/6], Step [700/1875], Custo: 0.1169, Acurácia(batch): 96.875%
Época [1/6], Step [800/1875], Custo: 0.0509, Acurácia(batch): 96.875%
Época [1/6], Step [900/1875], Custo: 0.0026, Acurácia(batch): 100.000%
Época [1/6], Step [1000/1875], Custo: 0.0399, Acurácia(batch): 100.000%
Época [1/6], Step [1100/1875], Custo: 0.0263, Acurácia(batch): 100.000%
Época [1/6], Step [1200/1875], Custo: 0.0046, Acurácia(batch): 100.000%
Época [1/6], Step [1300/1875], Custo: 0.0555, Acurácia(batch): 96.875%
Época [1/6], Step [1400/1875], Custo: 0.0105, Acurácia(batch): 100.000%
Época [1/

 Avaliação

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        outputs = model(images)
        predicted = outputs.argmax(dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Acurácia do Modelo em {total} imagens de teste: {100 * correct / total:.3f}%")